# Data Analysis
This file will contain all initial analysis of the data and plot it.

TODO: Chloe do this (https://github.com/vaquierm/RedditCommentTextClassification/issues/11)

In [286]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import nltk
from nltk import FreqDist
nltk.download('punkt')
from scipy.stats import spearmanr

import sys
sys.path.insert(0, '../../')

from utils import load_raw_training_data
from automate.create_feature_vectors import convert_all_raw_data_to_feature_vectors

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chloe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ImportError: cannot import name 'raw_data_dir_path' from 'src.main' (../..\src\main.py)

In [ ]:
def fetch_data(file_path):
    print("Reading data files...")
    data = pd.read_csv(file_path)
    
#todo: clean the data beforehand
    return data

In [252]:
def calculate_most_common_words(data, top_nb = 5):
    print("Computing most common words...")
    
    fdist_allsubs = FreqDist()
    list_subreddit_freq = []
    subreddit_names = data.subreddits.unique()
    
    for subreddit in subreddit_names:
        print("\t" + subreddit)
        
        # get data associated to subreddit
        subreddit_data = data.loc[data["subreddits"] == subreddit]
        
        # get frequency distribution of all words
        subreddit_comments = nltk.tokenize.word_tokenize(subreddit_data["comments"].str.cat(sep="\n"))
        fdist_subreddit = nltk.FreqDist(subreddit_comments)
        
#         fdist_subreddit.plot(5, cumulative = False)
#         plt.show()
        
        print(fdist_subreddit)
        print(fdist_subreddit.most_common(top_nb))
        list_subreddit_freq.append(fdist_subreddit)
        fdist_allsubs += fdist_subreddit
    
    print("\tall subreddits")
    print(fdist_allsubs)
    print(fdist_allsubs.most_common(top_nb))
    
    i = 0
    for subreddit_freq in list_subreddit_freq:
        idtf(fdist_allsubs, subreddit_freq, subreddit_names[i])
        i+=1

- IDF = nb of that word / total nb of that word
- Scoring function - which words are the most related to a subreddit
- Average number of words for comments
- Average number of words for each subreddit
- Average number of youtube links per comment in each subreddit category (probably high for like overwatch/league of legends)
- Check if there are any unknown charachters (There could be japanese charachters in comments in the anime subreddit)

In [266]:
def scoring_function(data):
#     todo: calculate scoring function based on the feature vectors
#     corr, _ = spearmanr(data, data2)
#     print('Spearmans correlation: %.3f' % corr)
    corr = data.corr()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
    fig.colorbar(cax)
    ticks = np.arange(0,len(data.columns),1)
    ax.set_xticks(ticks)
    plt.xticks(rotation=90)
    ax.set_yticks(ticks)
    ax.set_xticklabels(data.columns)
    ax.set_yticklabels(data.columns)
    plt.show()

In [261]:
def idtf(fdist_all, fdist_subreddit, subreddit_name):
    #using the frequency distribution we already have, find occurence of word in on subreddit / occurence of word in all
    words = fdist_all.keys()
    dictionary = {}
    for word in words:
        occurence_subreddit = fdist_subreddit[word]/fdist_all[word]
        if (occurence_subreddit > 0.5):
            dictionary[word] = occurence_subreddit
#             print("The word " + word + " occurs quite often in " + subreddit_name)
    sorted_dic = sorted(dictionary.items(), key=lambda kv: kv[1])
    print("The top 10 words for " + subreddit_name + ": \n\t")
    print(sorted_dic[-10:])

In [262]:
def calculate_avg_nb_words(data):
    COUNT_COLUMN = "count words"
    COUNT_YOUTUBE_COLUMN = "count youtube"
    
    print("Computing average number of words per comment...")
    data[COUNT_COLUMN] = data.apply(lambda row: len(row.comments), axis=1)
    count_youtube_comment = 0
    
    for subreddit in data.subreddits.unique():
        print("\t" + subreddit)
        # get data associated to subreddit
        subreddit_data = data.loc[data["subreddits"] == subreddit]
        
        # get mean of count of words 
        subreddit_mean_words = subreddit_data[COUNT_COLUMN].mean()
        print(subreddit_mean_words)
        
        # find the nb of youtube links and divide by nb of comments (if data is processed, can just get nb of "youtube")
        subreddit_count_youtube_comment = subreddit_data["comments"].str.count("youtube").sum()
        print("nb of youtube comment:" + str(subreddit_count_youtube_comment))
        #add to global youtube count
        count_youtube_comment += subreddit_count_youtube_comment
        
    print("\t all subreddits")
    # get mean of count of words 
    mean_words = data[COUNT_COLUMN].mean()
    print(mean_words)
    print("total nb of youtube comment:" + str(count_youtube_comment))


In [274]:
def calculate_statistics(data):
    print("Computing statistics...")
    
#     calculate_most_common_words(data)
#     calculate_avg_nb_words(data)
    scoring_function(convert_all_raw_data_to_feature_vectors())

In [275]:
data = fetch_data("../../data/raw_data/reddit_train.csv")
calculate_statistics(data)

Reading data files...
Computing statistics...


AttributeError: 'list' object has no attribute 'corr'